In [14]:
import torch
import requests
from io import BytesIO
from PIL import Image, ImageFile

from pkgs.openai.clip import load as load_model

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
device = 'cuda' 

In [15]:
def get_inputs(image, caption):
    captions     = processor.process_text(caption)
    pixel_values = processor.process_image(image.convert("RGB"))
    return captions['input_ids'].to(device), captions['attention_mask'].to(device), pixel_values.to(device).unsqueeze(0)

In [6]:
## pretrained = True loads the original OpenAI CLIP model trained on 400M image-text pairs
model, processor = load_model(name = 'RN50', pretrained = False)

100%|████████████████████████████████████████| 256M/256M [00:02<00:00, 116MiB/s]
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


In [ ]:
model.to(device)

In [9]:
## Replace with the location of the checkpoint 
## The link for checkpoints -- https://drive.google.com/drive/u/0/folders/1K0kPJZ3MA4KAdx3Fpq25dgW59wIf7M-x

checkpoint = '/content/drive/MyDrive/Spring22/checkpoints/cyclip.pt/best.pt'

In [ ]:
state_dict = torch.load(checkpoint, map_location = device)["state_dict"]
if(next(iter(state_dict.items()))[0].startswith("module")):
    state_dict = {key[len("module."):]: value for key, value in state_dict.items()}
model.load_state_dict(state_dict)
model.eval()  

In [16]:
url = 'https://images.pexels.com/photos/1108099/pexels-photo-1108099.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2'

In [17]:
response = requests.get(url)
img = Image.open(BytesIO(response.content))

In [26]:
caption1 = 'a photo of dogs'
caption2 = 'a photo of cats'

In [27]:
def clipscore(model, output):
  return (model.logit_scale.exp() * output.image_embeds @ output.text_embeds.t()).item()

In [28]:
input1 = get_inputs(img, caption1)
input2 = get_inputs(img, caption2)
output1 = model(input_ids = input1[0], attention_mask = input1[1], pixel_values = input1[2])
output2 = model(input_ids = input2[0], attention_mask = input2[1], pixel_values = input2[2])

print(clipscore(model, output1))
print(clipscore(model, output2))

41.97170639038086
34.910865783691406
